In [1]:
import yaml

from pkg.data.augmentation import build_augmentation
from pkg.training.optimizer import build_optimizer
from pkg.training.criterion import build_criterion
from pkg.utils.instantiate import instantiate
from pkg.training.trainer import Trainer

/home/cestari/.conda/envs/adni_rcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import yaml

path = "./test.yaml"

# Read configuration file
with open(path, 'r') as f:
    cfg = yaml.safe_load(f)

# Print configuration
print(cfg)

{'datamodule': {'class': 'pkg.data.datamodules.ADNIDataModule'}, 'dataset': {'class': 'pkg.data.datasets.ADNIDataset', 'args': {'scan_csv': '/project/aereditato/cestari/adni-mri-classification/data/preprocessing_multimodal/dataset_multimodal.csv', 'diagnostic_csv': '/project/aereditato/cestari/adni-mri-classification/data/preprocessing_multimodal/tables/DXSUM_02Feb2026.csv', 'modalities': {'MRI': ['MRI-T1-3T', 'MRI-T1-1.5T'], 'PET': ['PET-FDG']}, 'diagnosis': [1, 2, 3], 'tolerance': 180}}, 'split': {'mode': 'kfold', 'seed': 42, 'folds': 5, 'test_size': 0.1, 'val_size': 0.2}, 'loader': {'train': {'batch_size': 4, 'num_workers': 4, 'pin_memory': True, 'weighted_sampling': True}, 'eval': {'batch_size': 4, 'num_workers': 4, 'pin_memory': True}}, 'transform': {'train': [{'random_flip': {'p': 0.5}}, {'random_affine': {'p': 0.5, 'degrees': 5, 'translation': 5}}, {'random_noise': {'p': 0.2, 'std': 0.02}}], 'val': [], 'test': []}, 'model': {'class': 'pkg.models.model_resnet18.ResNet18', 'args':

In [3]:
# Dataset
ds = instantiate(cfg["dataset"])

# Data module 
dm = instantiate(cfg["datamodule"])

# Setup 
dm.setup(dataset=ds, 
         split=cfg["split"], 
         loader=cfg["loader"], 
         transform=cfg["transform"])

100%|██████████| 3818/3818 [00:26<00:00, 146.03it/s]


In [4]:
# Loop over folds
for fold in range(dm.n_folds()):

    # Set current fold
    dm.set_fold(fold)

    # Build model, optimizer, and criterion
    model = instantiate(cfg["model"])
    optim = build_optimizer(cfg["optimizer"], model_params=model.parameters())
    criterion = build_criterion(cfg["criterion"], train_labels=dm.train_labels)

    # Build and setup task
    task = instantiate(cfg["task"])
    task.setup(model, optim, criterion)

    # Callbacks 
    callbacks = instantiate(cfg["callbacks"])

    # Build trainer 
    trainer = Trainer(task, dm, callbacks, cfg["trainer"]["max_epochs"])

    print("Ok")
    # Fit model 
    #trainer.fit()
    
    # Load best model 
    
    # Test

    

    


ModuleNotFoundError: No module named 'metric'